# Netflix Text Analysis

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Load Data

In [2]:
fp: Path = Path("../../../../data/NETFLIX_REVIEWS.parquet")

data: pl.DataFrame = pl.read_parquet(source=fp)
data.head(3)

,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
i64,str,str,str,str,i64,i64,str,str
0,"""7e73f80e-a8fd-…","""15261855397701…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…"
1,"""dab55eca-c2a0-…","""23438294286543…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…"
2,"""a3b8fa06-8b8f-…","""17447360460835…","""A Google user""","""Works great on…",5,0,"""1.5.2 build 38…","""2011-05-12 18:…"


In [3]:
data = data.drop(columns=["", "pseudo_author_id", "author_name", "review_likes"])
data.head(3)

review_id,review_text,review_rating,author_app_version,review_timestamp
str,str,i64,str,str
"""7e73f80e-a8fd-…","""Works great on…",5,"""1.2.0 build 81…","""2011-05-12 18:…"
"""dab55eca-c2a0-…","""Works great on…",5,"""1.2.0 build 81…","""2011-05-12 18:…"
"""a3b8fa06-8b8f-…","""Works great on…",5,"""1.5.2 build 38…","""2011-05-12 18:…"


In [4]:
# Check if GPU is available
device: str = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cpu

### To Do

- Can you predict the review rating based on a review text?

In [5]:
from transformers import pipeline


TASK: str = "text-classification"
OUTPUT_DIR: str = "distilbert-base-uncased-finetuned-netflix-ratings"
MODEL_CHECKPOINT: str = f"chineidu/{OUTPUT_DIR}"
netflix_classifier: pipeline = pipeline(task=TASK, model=MODEL_CHECKPOINT)

2023-12-24 16:45:22.299242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
id2label: dict[str, Any] = {
    1: "very poor",
    2: "poor",
    3: "okay",
    4: "very good",
    5: "excellent",
}
label2id: dict[str, Any] = {
    "very poor": 1,
    "poor": 2,
    "okay": 3,
    "very good": 4,
    "excellent": 5,
}


def get_sentiment(reviews: list[str]) -> list[list[str], list[float]]:
    """This is used to obtain the label and the corresponding score."""
    result: list[dict[str, Any]] = netflix_classifier(reviews)
    labels: list[str] = [x.get("label") for x in result]
    ratings: list[str] = [label2id.get(x.get("label")) for x in result]

    return (labels, ratings)

In [7]:
texts: list[str] = data.select(pl.col("review_text")).to_series().to_list()[10:15]
texts

['Works great on Nexus S, but it needs subtitles support!',
 "Works on 3g even tho quality isn't the best but still...might be my new favorite app!",
 'Working great on Nexus S, picked up right where I left it watching.',
 'Works great. So far so good on Wifi. Nexus 1.',
 "Well I've tried it out on my Evo 4G and it works. Tried it through WiFi, and the quality is pretty decent.; it's very snappy. Then I tried it through 3G and I was thinking the worst, and while the quality is diminished; it still works. I don't live in an area with great signal but it works really well, though the pixelation can get bad at times. Haven't tried it on 4G but I'm expecting great results."]

In [8]:
netflix_classifier(texts)

[{'label': 'very good', 'score': 0.7804526686668396},
 {'label': 'very good', 'score': 0.4879053831100464},
 {'label': 'excellent', 'score': 0.8773263692855835},
 {'label': 'excellent', 'score': 0.7898459434509277},
 {'label': 'very good', 'score': 0.8486072421073914}]

In [9]:
# Actual
data.select(pl.col("review_rating")).to_series().to_list()[10:15]

[4, 5, 5, 5, 4]

In [10]:
# Predicted
get_sentiment(reviews=texts)

(['very good', 'very good', 'excellent', 'excellent', 'very good'],
 [4, 4, 5, 5, 4])

<br>

## Upsert Data To Qdrant

In [11]:
from sentence_transformers import SentenceTransformer


ENCODER_CHECKPOINT: str = "sentence-transformers/all-MiniLM-L6-v2"
# Load the model from HuggingFace Hub
encoder = SentenceTransformer(ENCODER_CHECKPOINT, device=device)

encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [12]:
encoder.get_max_seq_length(), encoder.get_sentence_embedding_dimension()

(256, 384)

In [13]:
import os
from getpass import getpass

from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())  # read local .env file
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

RUN_CELL: bool = False

if RUN_CELL:
    # Get API key without displaying it
    QDRANT_API_KEY: str = getpass("Please enter your API key: ")

In [14]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.exceptions import UnexpectedResponse


URL: str = "http://localhost:6333"
client = QdrantClient(
    url=URL,
    api_key=QDRANT_API_KEY,
)
DIMENSION: int = encoder.get_sentence_embedding_dimension()
METRIC: Any = models.Distance.COSINE
COLLECTION_NAME: str = "netflix-sentiment-analysis"

# Create a collection
try:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
            distance=METRIC,
        ),
        optimizers_config=models.OptimizersConfigDiff(indexing_threshold=0),
        shard_number=2,
    )
# If collection already exists
except UnexpectedResponse as err:
    print(err)

Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `netflix-sentiment-analysis` already exists!"},"time":0.000018871}'

In [15]:
# Get collection details
collection_info: Any = client.get_collection(collection_name=COLLECTION_NAME)
print(collection_info)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=775500,
    indexed_vectors_count=0,
    points_count=775500,
    segments_count=12,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None
            ),
            shard_number=2,
            sharding_method=None,
            replication_factor=1,
            write_consistency_factor=1,
            read_fan_out_factor=None,
            on_disk_payload=True,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=False,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=0,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None
    ),
    payload_schema={}
)

### Generate Embeddings

- Generate embeddings for all the reviews in the dataset.
- The embeddings, the sentiment label and score will be included in the Qdrant collection as metadata.
- The metadata will be used to understand the customer opinions.

In [16]:
print(data.head())

shape: (5, 5)
┌────────────────────┬────────────────────┬───────────────┬────────────────────┬───────────────────┐
│ review_id          ┆ review_text        ┆ review_rating ┆ author_app_version ┆ review_timestamp  │
│ ---                ┆ ---                ┆ ---           ┆ ---                ┆ ---               │
│ str                ┆ str                ┆ i64           ┆ str                ┆ str               │
╞════════════════════╪════════════════════╪═══════════════╪════════════════════╪═══════════════════╡
│ 7e73f80e-a8fd-4ff3 ┆ Works great on my  ┆ 5             ┆ 1.2.0 build        ┆ 2011-05-12        │
│ -b09b-502f0ad0…    ┆ Evo! Glad andr…    ┆               ┆ 819145-1.2.0-102   ┆ 18:50:37          │
│ dab55eca-c2a0-45a8 ┆ Works great on HTC ┆ 5             ┆ 1.2.0 build        ┆ 2011-05-12        │
│ -93e3-9860c1c5…    ┆ incredible. C…     ┆               ┆ 819145-1.2.0-102   ┆ 18:50:52          │
│ a3b8fa06-8b8f-4f2f ┆ Works great on     ┆ 5             ┆ 1.5.2 build 389    ┆ 2011-05-12        │
│ -a1fa-fd37c4cb…    ┆ nexus s            ┆               ┆                    ┆ 18:55:14          │
│ 837fdfa5-606d-4cec ┆ Working perfect    ┆ 5             ┆ 1.2.1 build        ┆ 2011-05-12        │
│ -9e9a-e4a83dad…    ┆ for me on EVO, r…  ┆               ┆ 843839-1.2.0-30    ┆ 19:31:46          │
│ a8aaecb2-6984-44f7 ┆ cm7 2.3.3 N1       ┆ 5             ┆ 1.5.2 build 389    ┆ 2011-05-12        │
│ -b958-3f89f64d…    ┆                    ┆               ┆                    ┆ 19:32:50          │
└────────────────────┴────────────────────┴───────────────┴────────────────────┴───────────────────┘

In [17]:
def retrieve_labels(ratings: list[int]) -> list[str]:
    """This is used to retrieve the labels."""
    labels: list[str] = [id2label.get(x) for x in ratings]

    return labels

In [18]:
retrieve_labels(ratings=[2, 3, 4, 5])

['poor', 'okay', 'very good', 'excellent']

In [19]:
data.select("review_timestamp").to_series().to_list()[0]

'2011-05-12 18:50:37'

In [20]:
from dateutil.parser import parse


def get_timestamp(dates: list[str]) -> list[float]:
    """This is used to convert the dates to timestamp."""
    timestamps: list[float] = [parse(d).timestamp() for d in dates]
    return timestamps

In [21]:
d: str = "2011-05-12 18:50:37"

get_timestamp(dates=[d])

[1305222637.0]

### Qdrant Upsert

- Using “Upsert” with Qdrant Vector Database [Blog](https://redandgreen.co.uk/qdrant-upsert/ai-ml/).
- [Official Documentation](https://qdrant.tech/documentation/concepts/points/#upload-points).


In [22]:
# Encode the entire data (Batch Upload)
import uuid

from tqdm.auto import tqdm


batch_size: int = 500
counter: int = 0

if RUN_CELL:
    for i in tqdm(range(0, len(data), batch_size)):

        # Find end of batch
        i_end: int = min(i + batch_size, data.shape[0])

        # Extract batch
        sample_rows: list[Any] = data.rows()[i:i_end]
        batch: pl.DataFrame = pl.DataFrame(
            sample_rows,
            schema=[
                "review_id",
                "review_text",
                "review_rating",
                "author_app_version",
                "review_date",  # renamed!
            ],
        )

        # IDs: list[str] = [str(uuid.uuid4()) for _ in range(batch.shape[0])]
        IDs: list[str] = batch.select(pl.col("review_id")).to_series().to_list()

        # Generate embeddings for batch
        batch_reviews: list[str] = (
            batch.select(pl.col("review_text")).to_series().to_list()
        )
        vectors: list[float] = encoder.encode(
            batch_reviews, show_progress_bar=False
        ).tolist()

        batch_ratings: list[str] = (
            batch.select(pl.col("review_rating")).to_series().to_list()
        )

        # Get sentiment label and score for reviews in the batch
        labels: list[str] = retrieve_labels(ratings=batch_ratings)

        # Add new column(s)
        batch: pl.DataFrame = pl.concat(
            [batch, pl.DataFrame({"label": labels})], how="horizontal"
        )
        dates: list[str] = batch.select(pl.col("review_date")).to_series().to_list()
        timestamp: list[float] = get_timestamp(dates=dates)
        timestamp_df: pl.DataFrame = pl.DataFrame(data={"timestamp": timestamp})
        batch: pl.DataFrame = pl.concat([batch, timestamp_df], how="horizontal")

        # Get metadata
        counter += batch_size
        if counter % 12_000 == 0:
            print(">>> [Creating payload] [Upserting data] <<<")
        payload_data: list[dict[str, Any]] = batch.to_dicts()

        # Upsert/insert these records to Qdrant
        _ = client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=IDs,
                payloads=payload_data,
                vectors=vectors,
            ),
        )

print(">>> [Done] <<<")

>>> [Done] <<<

### Note

- Due to the size of the data, I only upserted ~775.5k datapoints.
- I took ~1hr 58m to upsert.

<hr><br>

## Opinion Mining
- What are the reviewers saying about the app?
- Are there any patterns in sentiment over time or across different app versions?
- Automatically categorize reviews into different topics.
- What features are most indicative of a high/low review rating?

In [23]:
def count_sentiment(data: list[dict[str, Any]]) -> dict[str, int]:
    """This is used to count the sentiments."""
    from collections import Counter

    sentiments: list[str] = [x.dict().get("payload").get("label") for x in data]
    result: dict[str, int] = dict(Counter(sentiments))

    return result


import plotly.express as px


def visualize_result(sentiment: dict[str, Any], query: str) -> Any:

    try:
        vis_data: pd.DataFrame = (
            pd.DataFrame(sentiment, index=[0]).drop(columns=[None]).T
        )
    except KeyError:
        vis_data: pd.DataFrame = pd.DataFrame(sentiment, index=[0]).T

    vis_data = vis_data.reset_index(drop=False).rename(
        columns={0: "rating", "index": "label"}
    )
    fig = px.bar(
        vis_data,
        x="label",
        y="rating",
        labels=dict(index="rating", rating="count"),
        color="label",
        text_auto=True,
        title=query,
    )
    return fig.show()

### Q1 What are the reviewers saying about the app?

- e.g. are the customers satisfied with the app?
- do you like it? 

In [24]:
N: int = 5_000
query: str = "are the customers satisfied with the app?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,
    limit=N,
)
# for hit in hits:
# print(hit.payload, "score:", hit.score)

print(hits[:5])

[
    ScoredPoint(
        id='bac9519b-0005-420d-970e-3ef515f1b3a5',
        version=369,
        score=0.84076256,
        payload={
            'author_app_version': '3.5.1 build 1566',
            'label': 'very good',
            'review_date': '2014-06-11 03:56:09',
            'review_id': 'bac9519b-0005-420d-970e-3ef515f1b3a5',
            'review_rating': 4,
            'review_text': 'This app is very nice. I mostly use it to entertain the kids when we are out and about,
but it has never given me any serious problems in that respect. From MY experiences ALONE, I can say I am a 
satisfied customers thus far.',
            'timestamp': 1402455369.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='af768543-c619-4486-b8b6-e6ae5581bafb',
        version=1202,
        score=0.723863,
        payload={
            'author_app_version': '5.16.1 build 26734',
            'label': 'poor',
            'review_date': '2018-06-19 06:09:06',
            'review_id': 'af768543-c619-4486-b8b6-e6ae5581bafb',
            'review_rating': 2,
            'review_text': 'Very good app, but totally unsatisfactory customer service. Please teach \nyour 
customer service reps to talk to customers with respect if we are \nhaving an issue that has to be resolved.',
            'timestamp': 1529384946.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='62c2478d-7ea7-4286-9614-efaa64b950cb',
        version=83,
        score=0.71924067,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2012-05-22 16:01:13',
            'review_id': '62c2478d-7ea7-4286-9614-efaa64b950cb',
            'review_rating': 5,
            'review_text': "I've been a customer for more than 2 years great app",
            'timestamp': 1337698873.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='513ebf2b-af2d-4436-9455-30e404ee530b',
        version=1399,
        score=0.7089151,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2019-03-03 19:44:08',
            'review_id': '513ebf2b-af2d-4436-9455-30e404ee530b',
            'review_rating': 5,
            'review_text': 'good app and customer service when needed',
            'timestamp': 1551638648.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='434e85d0-a6cc-4355-bc1a-3614f0574aa8',
        version=863,
        score=0.7002642,
        payload={
            'author_app_version': '4.13.2 build 14565',
            'label': 'excellent',
            'review_date': '2017-02-27 13:36:14',
            'review_id': '434e85d0-a6cc-4355-bc1a-3614f0574aa8',
            'review_rating': 5,
            'review_text': 'Have no complaints about app',
            'timestamp': 1488198974.0
        },
        vector=None,
        shard_key=None
    )
]

In [25]:
sentiment: dict[str, int] = count_sentiment(hits)
sentiment

{'very good': 684,
 'poor': 92,
 'excellent': 3767,
 'okay': 225,
 'very poor': 231,
 None: 1}

In [26]:
visualize_result(sentiment=sentiment, query=query)

In [27]:
# N: int = 2
query: str = "do you like it?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,
    limit=N,
)

print(hits[:5])
sentiment: dict[str, int] = count_sentiment(hits)
visualize_result(sentiment=sentiment, query=query)

[
    ScoredPoint(
        id='1fd5cc16-353d-488f-b09a-c67b895348ab',
        version=1309,
        score=0.7752483,
        payload={
            'author_app_version': '3.16.3 build 5359',
            'label': 'excellent',
            'review_date': '2018-11-11 14:53:02',
            'review_id': '1fd5cc16-353d-488f-b09a-c67b895348ab',
            'review_rating': 5,
            'review_text': 'I love it?',
            'timestamp': 1541944382.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='a28ea87f-eb1d-4261-b070-4295dfdee24f',
        version=1399,
        score=0.74413604,
        payload={
            'author_app_version': '6.24.0 build 12 31651',
            'label': 'excellent',
            'review_date': '2019-03-04 02:56:52',
            'review_id': 'a28ea87f-eb1d-4261-b070-4295dfdee24f',
            'review_rating': 5,
            'review_text': 'I like it ?',
            'timestamp': 1551664612.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='e741c76d-daba-4308-a424-03f703c2dedd',
        version=1244,
        score=0.7029816,
        payload={
            'author_app_version': '6.9.1 build 29147',
            'label': 'excellent',
            'review_date': '2018-08-22 02:41:04',
            'review_id': 'e741c76d-daba-4308-a424-03f703c2dedd',
            'review_rating': 5,
            'review_text': 'Love it?',
            'timestamp': 1534902064.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='ca435235-92aa-4c3c-8177-1643644ff7e2',
        version=462,
        score=0.70298153,
        payload={
            'author_app_version': '',
            'label': 'very good',
            'review_date': '2014-12-16 01:56:24',
            'review_id': 'ca435235-92aa-4c3c-8177-1643644ff7e2',
            'review_rating': 4,
            'review_text': 'Love it ?',
            'timestamp': 1418691384.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='b6528ee2-0e7c-4d24-834f-d2c4bbb3615e',
        version=868,
        score=0.7003145,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-03-05 10:32:03',
            'review_id': 'b6528ee2-0e7c-4d24-834f-d2c4bbb3615e',
            'review_rating': 5,
            'review_text': 'Yes I like it',
            'timestamp': 1488706323.0
        },
        vector=None,
        shard_key=None
    )
]

### Q2 Are there any patterns in sentiment over time? 

- e.g. between Jan 2018 to Jan 2019

In [28]:
data.select(pl.col("review_timestamp")).describe()

describe,review_timestamp
str,str
"""count""","""1531126"""
"""null_count""","""0"""
"""mean""",null
"""std""",null
"""min""","""2011-05-12 18:…"
"""25%""",null
"""50%""",null
"""75%""",null
"""max""","""2023-11-15 22:…"


In [29]:
get_timestamp(["2018-01-01 00:00:00"])[0]

1514761200.0

In [30]:
N: int = 8_000
query: str = "are the customers satisfied with the app?"
query_vector: list[float] = encoder.encode(query).tolist()

start_date, stop_date = [
    "2018-01-01 00:00:00",
    "2019-01-01 00:00:00",
]  # yyyy-mm-dd HH:MM:SS
start_time: float = get_timestamp([start_date])[0]
stop_time: float = get_timestamp([stop_date])[0]

query_filter: dict[str, Any] = {
    "must": [
        {"key": "timestamp", "range": {"gte": start_time, "lte": stop_time}},
    ]
}

hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=query_filter,
    limit=N,
)

print(hits[:15])

[
    ScoredPoint(
        id='af768543-c619-4486-b8b6-e6ae5581bafb',
        version=1202,
        score=0.723863,
        payload={
            'author_app_version': '5.16.1 build 26734',
            'label': 'poor',
            'review_date': '2018-06-19 06:09:06',
            'review_id': 'af768543-c619-4486-b8b6-e6ae5581bafb',
            'review_rating': 2,
            'review_text': 'Very good app, but totally unsatisfactory customer service. Please teach \nyour 
customer service reps to talk to customers with respect if we are \nhaving an issue that has to be resolved.',
            'timestamp': 1529384946.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='d83d6e85-4a06-43f8-94e3-0145db726f13',
        version=1097,
        score=0.69769436,
        payload={
            'author_app_version': '1.0',
            'label': 'excellent',
            'review_date': '2018-01-02 02:03:57',
            'review_id': 'd83d6e85-4a06-43f8-94e3-0145db726f13',
            'review_rating': 5,
            'review_text': 'Is. Satisfied  with  the  easygoing  apps',
            'timestamp': 1514855037.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='188b46f8-6bbf-494a-ab18-76453b8e06aa',
        version=1230,
        score=0.67664903,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-07-31 04:33:39',
            'review_id': '188b46f8-6bbf-494a-ab18-76453b8e06aa',
            'review_rating': 5,
            'review_text': 'Is app is good',
            'timestamp': 1533008019.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='8f7e2c1a-f170-406b-a20d-7695216c727a',
        version=1109,
        score=0.65183043,
        payload={
            'author_app_version': '',
            'label': 'very good',
            'review_date': '2018-01-21 03:51:15',
            'review_id': '8f7e2c1a-f170-406b-a20d-7695216c727a',
            'review_rating': 4,
            'review_text': 'Is a good app',
            'timestamp': 1516503075.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='5d2129e9-2616-4635-9c93-a6be9dffe7db',
        version=1111,
        score=0.65183043,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-01-24 03:52:41',
            'review_id': '5d2129e9-2616-4635-9c93-a6be9dffe7db',
            'review_rating': 5,
            'review_text': 'is a good app',
            'timestamp': 1516762361.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='13982fe1-463e-407f-88f5-79fcd55a2c8a',
        version=1173,
        score=0.65183043,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-05-05 01:23:02',
            'review_id': '13982fe1-463e-407f-88f5-79fcd55a2c8a',
            'review_rating': 5,
            'review_text': 'Is a good App',
            'timestamp': 1525479782.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='745cd3ed-ce1d-4373-8762-0dc9477c227d',
        version=1101,
        score=0.65183043,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-01-07 18:09:43',
            'review_id': '745cd3ed-ce1d-4373-8762-0dc9477c227d',
            'review_rating': 5,
            'review_text': 'Is a good app',
            'timestamp': 1515344983.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='47e346d0-37c4-438e-be76-2db8317972e0',
        version=1294,
        score=0.65183043,
        payload={
            'author_app_version': '6.11.0 build 29552',
            'label': 'excellent',
            'review_date': '2018-10-28 16:55:39',
            'review_id': '

In [31]:
sentiment: dict[str, int] = count_sentiment(hits)
visualize_result(sentiment=sentiment, query=query)

### Q3 Automatically categorize reviews into different topics

- Topic Modelling

In [32]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS
from umap import UMAP


stopwords: list[str] = list(STOP_WORDS)

# Step 1
# embedding_model: SentenceTransformer = SentenceTransformer(encoder_checkpoint)

# Step 2
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)

# Step 3
hdbscan_model = HDBSCAN(
    max_cluster_size=100,
    prediction_data=True,  # Whether to generate extra cached data for predicting labels
    gen_min_span_tree=True,  # Whether to generate the min spanning tree with regard to mutual reachability distance for later analysis.
)

# Step 4
# Add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(stop_words=stopwords)

# Step 5
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

In [33]:
model = BERTopic(
    nr_topics=5,
    embedding_model=encoder,  # Step 1 - Extract embeddings
    umap_model=umap_model,  # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,  # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
    top_n_words=5,
    language="english",
    calculate_probabilities=True,
    verbose=True,
)

model

In [34]:
reviews: list[str] = data.select(pl.col("review_text")).to_series().to_list()
reviews[15:20]

['Works for me, Evo 4G on CM7 with Android 2.3.3 over Sprint EVDO',
 'It tries to work on my G2 with CM7-nightly. But movies stop streaming after the first minute.',
 "2 stars just for supporting my G2. It may be related to the ROM I'm using, but when I try to play a video it just has a white screen. I can hear sound, but no video. I'm using a version of the leaked Gingerbread update.",
 'Works with no issue on Stock G2. Sweet!',
 'Works great on my Nexus One with cm7.0.3']

In [35]:
# It takes approx 2m 20s
topics, probs = model.fit_transform(documents=reviews[:10_000])

2023-12-24 16:45:56,529 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2023-12-24 16:46:28,261 - BERTopic - Embedding - Completed ✓
2023-12-24 16:46:28,262 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-24 16:46:57,584 - BERTopic - Dimensionality - Completed ✓
2023-12-24 16:46:57,586 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-24 16:48:33,717 - BERTopic - Cluster - Completed ✓
2023-12-24 16:48:33,718 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-24 16:48:53,708 - BERTopic - Representation - Completed ✓
2023-12-24 16:48:53,709 - BERTopic - Topic reduction - Reducing number of topics
2023-12-24 16:48:56,969 - BERTopic - Topic reduction - Reduced number of topics from 604 to 5


In [36]:
model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1720,-1_evo4g_evo_htc_android,"[evo4g, evo, htc, android, droid]","[Finally. Works great evo., Droid X... I love the app! It works GREAT! I'm always on the go! Finally!!!, Love this works great on my EVO]"
1,0,7613,0_evo_4g_htc_3g,"[evo, 4g, htc, 3g, nexus]","[Works great. Evo 4g, Works great- HTC evo, Works great! HTC EVO 4G]"
2,1,606,1_awesome_awsome_love_fantastic,"[awesome, awsome, love, fantastic, amazing]","[!!! LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!LOVE IT!!!!, Awesome! Love it :), LOVE IT!!!! FINALLY!!!!!!]"
3,2,53,2_netflix_rate_prices_subscription,"[netflix, rate, prices, subscription, increase]","[Screw you and your massive price increase. High prices for a small selection of streaming content that isn't even HD. FU., Raising price 60%, this is my last month, Too bad we will soon be canceling our Netflix service because of your 60% price increase! :(]"
4,3,8,3_como_spanish_malisimo_gusta,"[como, spanish, malisimo, gusta, esta]","[Me gusta los huevos., No me esta sirbiendo como deberia, Esta maldita Esta application ,pasen la voz.....]"


In [37]:
model.visualize_barchart(
    top_n_topics=10,
    width=300,
    height=300,
)